######################################################################################################################
######################################### LOAD BREAKED ECG'S #########################################################
######################################################################################################################


In [2]:
def breaked_ECGs():
    """ECG's with breakes (empties in ECG line)"""

    # Id's of breacked first ecg's
    breaked_first_ecg_ids = []
    # Id's of breacked second ecg's
    breaked_second_ecg_ids = []

    with open('breaked_list/breaked_first_ecg.txt', 'r', encoding='utf-8') as file:
        for str in file:
            breaked_first_ecg_ids.append(str.strip())  # strip - remove spaces and '\n'

    with open('breaked_list/breaked_second_ecg.txt', 'r', encoding='utf-8') as file:
        for str in file:
            breaked_second_ecg_ids.append(str.strip())

    print("Breaked_first_ecg_ids: ", breaked_first_ecg_ids)
    print("Breaked_second_ecg_ids: ", breaked_second_ecg_ids)

    return breaked_first_ecg_ids, breaked_second_ecg_ids

def sets_with_breaked_ECGs(breaked_first_ecg_ids, breaked_second_ecg_ids):
    """General for two ECG's and unique of each ECG"""

    # General for two lists
    general = list(set(breaked_first_ecg_ids) & set(breaked_second_ecg_ids))  # Пересечение множеств
    general.sort()
    print("General: ", general)

    first_unique = list(set(breaked_first_ecg_ids) - set(general))  # First unique from general
    second_unique = list(set(breaked_second_ecg_ids) - set(general))  # Second unique from general
    first_unique.sort()
    second_unique.sort()
    print("First unique: ", first_unique)
    print("Second unique: ", second_unique)

    return general, first_unique, second_unique

In [ ]:
path_to_dataset_folder = 'D:/SCIENCE/Datasets/autonomic-aging-a-dataset-to-quantify-changes-of-cardiovascular-autonomic-function-during-healthy-aging-1.0.0'
csv_info_file = 'subject-info.csv'

DATABASE_ATTRIBUTES = []
DATABASE = {}

def print_database_attributes():
    print(f'Column names are: {", ".join(DATABASE_ATTRIBUTES)}')


class RECORD:

    """ Record class that represents necessary information about ECG database """

    # Database with filtered ages

    def __init__(self, id, age_group, sex, bmi, length, device, ecg_1, ecg_2):

        """Without third time series - pressure"""

        self.Id = id
        self.AgeGroup = age_group
        self.Sex = sex
        self.BMI = bmi
        self.Length = length
        self.Device = device
        #self.ECG_1 = ecg_1
        #self.ECG_2 = ecg_2

        DATABASE[id] = self


    def __str__(self):

        self.plot()
        return str(f'\tId: {self.Id:>6};    Age_group: {age_groups[self.AgeGroup]:>9};    Sex: {gender[self.Sex]:>6};    BMI: {self.BMI:>6};    Length: {self.Length:>5};    Device: {device[self.Device]}.')

    def plot(self):

        plt.plot(self.ECG_1)
        plt.ylabel("mV")
        plt.show()
        #wfdb.plot_wfdb(record, title='Record ' + id + ' from Physionet Autonomic ECG')
# ECG_dictionary with information about ECG



def print_database():
    for id in DATABASE.keys():
       print(DATABASE[id])



def open_record(id, min_point, max_point, remotely):

    """ Open each record with ECGs by Id

        Input parameters:
            - Id - id of record
            - min_point - minimum point, at which starts ECG (including this point)
            - max_point - maximum point, at which ends ECG (not including this point)

        Output parameters:
            - [sequence_1, sequence_2] - list with sequence_1 for first ECG and sequence_2 for second ECG

            Describing:
                wfdb.rdrecord(path + '/' + id, min_point, max_point, [0, 1])

                min_point = 0 - The starting sample number to read for all channels
                                (point from what graphic starts (min_point)).

                max_point = None - The sample number at which to stop reading for all
                channels (max_point). Reads the entire duration by default.

                [0, 1] - first two channels (ECG 1, ECG 2); [0] - only first ECG.
            """

    record = None

    if min_point < 0:
        print("Too low minimal point of ECG! Now minimal point is 0!")
        min_point = 0

    if os.path.isfile(path_to_dataset_folder + '/' + id + '.hea') or os.path.isfile(path_to_dataset_folder + '/' + id + '.dat'):
        try:
            record = open_record_wfdb(id, min_point, max_point, remotely)

        except:
            max_point = None
            record = open_record_wfdb(id, min_point, max_point, remotely)
            print("Too hight maximal point of ECG! Now maximal point is None!")
    else:
        print("File with record doesn't exist!")
        return None

    #display(record.__dict__)


    sequence_1 = []
    sequence_2 = []


    # print(record.p_signal)

    for x in record.p_signal:

        # Use first ECG
        sequence_1.append(x[0])

        # Use second ECG
        sequence_2.append(x[1])


    print("Length of first ECG with id {0}: {1}".format(id, str(len(sequence_1))))
    print("Length of second ECG with id {0}: {1}".format(id, str(len(sequence_2))))
    #print(sequence)

    create_breaks_file(sequence_1, sequence_2)
    #wfdb.plot_wfdb(record, channels=[0], title='Record ' + id + ' from Physionet Autonomic ECG')

    # Проверяем, есть ли сигнал
    if record.p_signal is not None:
        # Получаем временные метки в миллисекундах
        fs = record.fs  # Частота дискретизации
        num_samples = record.p_signal.shape[0]
        time_ms = [(i / fs) * 1000 for i in range(num_samples)]  # Переводим в миллисекунды

        # Рисуем сигнал вручную
        plt.figure(figsize=(10, 4))
        plt.plot(time_ms, record.p_signal[:, 0])  # Только первый канал
        plt.xlabel("Time (ms)")
        plt.ylabel("Amplitude (mV)")
        plt.title("ECG signal")
        

def read_ECGs_annotation_data(is_remotely, except_breaked):

        """ Open csv info file, print header and information for each record.

            input: is_remotely - download annotation file and record remotely from internet

        """

        # Path to CSV file with annotation
        path=""

        # Id's of breaked first and second ecg's
        breaked_first_ecg_ids, breaked_second_ecg_ids = breaked_ECGs()

        # Id's general both for first ecg, first unique, second unique
        general, first_unique, second_unique = sets_with_breaked_ECGs(breaked_first_ecg_ids, breaked_second_ecg_ids)

        # Check, if dataset is remotely located
        if is_remotely:
            path = csv_info_file
        else:
            path = path_to_dataset_folder + '/' + csv_info_file

        with open(path) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')

            # Получаем первую строку для инициализации DATABASE_ATTRIBUTES
            first_row = next(csv_reader)

            # Setting counter for first row
            line_count = 0

            for col in first_row:
                DATABASE_ATTRIBUTES.append(col)

            # Union of general and first_unique (breaked first ECG)
            breaked_first_ecg = list(set(general) | set(first_unique))
            breaked_first_ecg.sort()

            # Обрабатываем оставшиеся строки
            for row in csv_reader:

                line_count += 1

                # Check, if ECG in first ecg breaked list
                if (row[0] in breaked_first_ecg):
                    continue

                # 780 - 800; 1081 <
                #if (line_count < 1081):
                #    continue

                # If Id is not available
                if (row[0] == 'NaN'):
                    continue

                # If age category is not available. For future maybe do self-organizing without ages.
                if (row[1] == 'NaN'):
                    continue

                else:
                    # Open record returns ecg_1 and ecg_2
                    ecg_s = open_record(row[0], 0, None, remotely=is_remotely)

                    if ecg_s is None:
                        continue

                    # Signal with first ECG
                    signal = ecg_s[0]

                    # Частота дискретизації
                    sampling_rate = 1000

                    r_peaks, rr_intervals = extract_RR_intervals_time_series_and_plot_them(signal, sampling_rate, row[0])

                    